In [1]:
import datetime
import glob
import os

from herbie import Herbie
import numpy as np
import pandas as pd
from pvlib import location, solarposition, irradiance

import sys
sys.path.insert(0, os.path.join("..", "AMY"))
import constants, utils

latitude = 29.25,  # N
longitude = 360-98.31,  # W


In [14]:
data_dir = "/Users/jeh/Library/CloudStorage/OneDrive-TheUniversityofTexasatAustin/business/UTexas/projects/AMY/cache"

data_files = glob.glob(os.path.join(data_dir, "*.csv"))
data_files.sort()

dfs = []
for data_file in data_files:
    dfs.append(pd.read_csv(data_file, index_col=0))
df = pd.concat(dfs)
df.index = pd.to_datetime(df.index)
df

,tcc,unknown,pwat,tc_mdens,gh,u10,v10,t2m,d2m,r2,...,vbdsf,vddsf,ltng,prate,tp,crain,cfrzr,cicep,csnow,cpofp
2021-12-31 19:00:00,81.0,0.0,30.381283,5.000000e-08,798.5615,0.740710,3.719895,298.30490,294.54865,80.0,...,310.0,188.1,0.0,0.0,0.007,0.0,0.0,0.0,0.0,-50.0
2021-12-31 20:00:00,4.0,0.0,29.937360,4.000000e-08,NaN,0.064947,5.198508,300.31995,293.49902,66.2,...,840.0,98.6,0.0,0.0,0.000,0.0,0.0,0.0,0.0,-50.0
2021-12-31 21:00:00,0.0,0.0,30.286688,4.000000e-08,NaN,-0.717142,4.615349,301.48914,292.29065,57.5,...,784.0,88.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,-50.0
2021-12-31 22:00:00,0.0,0.0,30.371185,5.000000e-08,NaN,-0.566741,4.025953,301.36115,292.08496,57.1,...,673.0,70.1,0.0,0.0,0.000,0.0,0.0,0.0,0.0,-50.0
2021-12-31 23:00:00,4.0,0.0,31.071861,4.600000e-08,NaN,-1.730537,2.449102,300.05980,293.78275,65.3,...,352.2,43.9,0.0,0.0,0.000,0.0,0.0,0.0,0.0,-50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01 20:00:00,0.0,0.0,43.355343,5.400000e-06,NaN,-2.657869,2.929174,311.40115,291.55487,33.9,...,920.0,84.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,-50.0
2023-08-01 21:00:00,27.0,0.0,45.396960,5.500000e-06,NaN,-1.694483,2.965555,312.67258,290.03270,28.6,...,892.0,82.6,0.0,0.0,0.000,0.0,0.0,0.0,0.0,-50.0
2023-08-01 22:00:00,0.0,0.0,44.479767,6.300000e-06,NaN,-2.854485,3.532358,311.88403,290.85724,30.9,...,839.0,82.1,0.0,0.0,0.000,0.0,0.0,0.0,0.0,-50.0
2023-08-01 23:00:00,5.0,0.0,44.671688,8.300000e-06,NaN,-4.072405,1.594299,312.95328,288.56230,25.3,...,754.0,73.8,0.0,0.0,0.000,0.0,0.0,0.0,0.0,-50.0


In [15]:
# TODO: Validate that all hours are there

In [16]:
# intermediate calculations
## Opaque Sky Cover
df["opaque_sky_cover"] = utils.cloud_cover_to_opaque_sky_cover(
    df["lcc"].values, df["mcc"].values, df["hcc"].values, df["tcc"].values
)
## Wind Projection
u,v = utils.get_coordinate_projections(latitude, longitude)
df["wind_E"] = df["u10"] * u[0] + df["v10"] * v[0]
df["wind_N"] = df["u10"] * u[1] + df["v10"] * v[1]
## Snow Days
df["snow_day"] = df.groupby(df.index.date)["csnow"].transform("sum") > 0
snow_days = df[df["snow_day"]].copy()
snow_days["last_snow_day"] = snow_days.index
df = pd.merge_asof(df, snow_days['last_snow_day'],
                   left_index=True, right_index=True,
                   direction='backward')

df['Days Since Last Snowfall [Days]'] = (df.index-df['last_snow_day']).dt.days
## Solar Position
detailed_times = pd.date_range(df.index.min(), df.index.max(), freq="5min")
san_antonio = location.Location(latitude=29.25, longitude=360-98.31, name="San Antonio")
sp_5min = solarposition.get_solarposition(detailed_times, san_antonio.latitude, san_antonio.longitude)

sp_5min["extraterrestrial_direct_normal_radiation"] = \
    utils.get_extraterrestrial_direct_normal_radiation(sp_5min.index.dayofyear)
sp_5min["extraterrestrial_horizontal_radiation"] = \
    utils.get_extraterrestrial_horizontal_radiation(sp_5min["apparent_zenith"], 
        sp_5min["extraterrestrial_direct_normal_radiation"])
    
sp = sp_5min.resample("1H").mean()
sp

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2022-Jan-01 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


/Users/jeh/anaconda3/envs/amy/lib/python3.8/site-packages/herbie/core.py:797: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  logic = df.search_this.str.contains(searchString)


,apparent_zenith,zenith,apparent_elevation,elevation,azimuth,equation_of_time,extraterrestrial_direct_normal_radiation,extraterrestrial_horizontal_radiation
2021-12-31 19:00:00,53.839123,53.862121,36.160877,36.137879,194.534023,-3.207968,1405.913000,829.451064
2021-12-31 20:00:00,58.799783,58.827575,31.200217,31.172425,209.989326,-3.227787,1405.913000,727.914084
2021-12-31 21:00:00,66.560377,66.599311,23.439623,23.400689,222.828012,-3.247597,1405.913000,558.693642
2021-12-31 22:00:00,76.251889,76.321729,13.748111,13.678271,233.168114,-3.267398,1405.913000,333.665315
2021-12-31 23:00:00,87.120620,87.335159,2.879380,2.664841,241.575366,-3.287192,1405.913000,70.512784
...,...,...,...,...,...,...,...,...
2023-08-01 20:00:00,27.145201,27.153872,62.854799,62.846128,250.854616,-6.356187,1322.136455,1174.244949
2023-08-01 21:00:00,39.872930,39.887091,50.127070,50.112909,262.923498,-6.353411,1322.136455,1012.544449
2023-08-01 22:00:00,52.919566,52.942056,37.080434,37.057944,271.006937,-6.350618,1322.136455,795.446505
2023-08-01 23:00:00,65.941688,65.980001,24.058312,24.019999,277.799140,-6.347807,1322.136455,537.852399


In [17]:
df

,tcc,unknown,pwat,tc_mdens,gh,u10,v10,t2m,d2m,r2,...,cfrzr,cicep,csnow,cpofp,opaque_sky_cover,wind_E,wind_N,snow_day,last_snow_day,Days Since Last Snowfall [Days]
2021-12-31 19:00:00,81.0,0.0,30.381283,5.000000e-08,798.5615,0.740710,3.719895,298.30490,294.54865,80.0,...,0.0,0.0,0.0,-50.0,81.0,0.703666,3.725323,False,NaT,NaN
2021-12-31 20:00:00,4.0,0.0,29.937360,4.000000e-08,NaN,0.064947,5.198508,300.31995,293.49902,66.2,...,0.0,0.0,0.0,-50.0,4.0,0.013206,5.198743,False,NaT,NaN
2021-12-31 21:00:00,0.0,0.0,30.286688,4.000000e-08,NaN,-0.717142,4.615349,301.48914,292.29065,57.5,...,0.0,0.0,0.0,-50.0,0.0,-0.763057,4.609687,False,NaT,NaN
2021-12-31 22:00:00,0.0,0.0,30.371185,5.000000e-08,NaN,-0.566741,4.025953,301.36115,292.08496,57.1,...,0.0,0.0,0.0,-50.0,0.0,-0.606794,4.021460,False,NaT,NaN
2021-12-31 23:00:00,4.0,0.0,31.071861,4.600000e-08,NaN,-1.730537,2.449102,300.05980,293.78275,65.3,...,0.0,0.0,0.0,-50.0,4.0,-1.754863,2.435870,False,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01 20:00:00,0.0,0.0,43.355343,5.400000e-06,NaN,-2.657869,2.929174,311.40115,291.55487,33.9,...,0.0,0.0,0.0,-50.0,0.0,-2.686946,2.908892,False,2023-02-01 23:00:00,180.0
2023-08-01 21:00:00,27.0,0.0,45.396960,5.500000e-06,NaN,-1.694483,2.965555,312.67258,290.03270,28.6,...,0.0,0.0,0.0,-50.0,27.0,-1.723949,2.952570,False,2023-02-01 23:00:00,180.0
2023-08-01 22:00:00,0.0,0.0,44.479767,6.300000e-06,NaN,-2.854485,3.532358,311.88403,290.85724,30.9,...,0.0,0.0,0.0,-50.0,0.0,-2.889559,3.510557,False,2023-02-01 23:00:00,180.0
2023-08-01 23:00:00,5.0,0.0,44.671688,8.300000e-06,NaN,-4.072405,1.594299,312.95328,288.56230,25.3,...,0.0,0.0,0.0,-50.0,5.0,-4.088156,1.563367,False,2023-02-01 23:00:00,181.0


In [18]:
date_range = pd.date_range(df.index.values[0], df.index.values[-1], freq="1H")
[i for i in date_range if i not in df.index.values]


[]

In [7]:
amy_df = pd.DataFrame()
amy_df["year"] = df.index.year
amy_df["month"] = df.index.month
amy_df["day"] = df.index.day
amy_df["hour"] = df.index.hour
amy_df["minute"] = df.index.minute
amy_df["data_flags"] = "NOAA HRRR"
amy_df["dry bulb temperature [C]"] = df["t2m"].values - 273.15
amy_df["dew point temperature [C]"] = df["d2m"].values - 273.15
amy_df["relative humidity [%]"] = df["r2"].values
amy_df["atmospheric station pressure [Pa]"] = df["sp"].values
amy_df["extraterrestrial horizontal radiation [Wh/m^2]"] = sp["extraterrestrial_horizontal_radiation"].values
amy_df["extraterrestrial direct normal radiation [Wh/m^2]"] = sp["extraterrestrial_direct_normal_radiation"].values
amy_df["horizontal infrared radiation intensity [Wh/m^2]"] = utils.horizontal_ir(
    df["t2m"].values, df["d2m"].values, df["opaque_sky_cover"].values/10)
amy_df["global horizontal radiation [Wh/m^2]"] = df["vbdsf"].values + df["vddsf"].values
amy_df["direct normal radiation [Wh/m^2]"] = df["vddsf"].values
amy_df["diffuse horizontal radiation [Wh/m^2]"] = df["vbdsf"].values
amy_df["global horizontal illuminance [lux]"] = utils.solar_irradiance_to_lux(df["vbdsf"].values + df["vddsf"].values)
amy_df["direct normal illuminance [lux]"] = utils.solar_irradiance_to_lux(df["vddsf"].values)
amy_df["diffuse horizontal illuminance [lux]"] = utils.solar_irradiance_to_lux(df["vbdsf"].values)
amy_df["zenith luminance [cd/m^2]"] = utils.solar_irradiance_to_lux(
    df["vbdsf"] * np.cos(np.deg2rad(sp["zenith"].values)) + df["vddsf"].values)
amy_df["wind direction [deg]"] = df.apply(lambda x: utils.get_wind_direction(x["wind_E"], x["wind_N"]), axis=1)
amy_df["wind speed [m/s]"] = utils.get_wind_speed(df["wind_E"].values, df["wind_N"].values)
amy_df["total sky cover [tenths]"] = df["tcc"].values / 10
amy_df["opaque sky cover [tenths]"] = df["opaque_sky_cover"].values / 10
amy_df["visibility [km]"] = df["vis"].values / 1000
amy_df["ceiling height [m]"] = df["gh"].values 
amy_df["present weather observation"] = 0 ##########
amy_df["present weather codes"] = df.apply(
    lambda x: utils.parse_weather_code(
        accumulated_precipitation = x["tp"],
        freezing_rain=x["cfrzr"],
        ice_pellets=x["cicep"],
        lightning=x["ltng"],
        rain=x["crain"],
        snow=x["csnow"],
        pct_frozen_precipitation=x["cpofp"],
        visibility=x["vis"],
        wind_gust_speed=x["gust"],
        smoke=x["tc_mdens"]
    ), axis=1).values
amy_df["precipitable water [mm]"] = df["pwat"].values
amy_df["aerosol optical depth [thousandths]"] = df["unknown"].values
amy_df["snow depth [cm]"] = df["sde"].values * 100
amy_df["days since last snowfall"] = df['Days Since Last Snowfall [Days]'].values
amy_df["albedo [nondim]"] = utils.get_albedo(latitude, longitude)
amy_df["liquid precipitation depth [mm]"] = df["tp"].values
amy_df["liquid precipitation quantity [hr]"] = 1
amy_df

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2022-Jan-01 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


,year,month,day,hour,minute,data_flags,dry bulb temperature [C],dew point temperature [C],relative humidity [%],atmospheric station pressure [Pa],...,ceiling height [m],present weather observation,present weather codes,precipitable water [mm],aerosol optical depth [thousandths],snow depth [cm],days since last snowfall,albedo [nondim],liquid precipitation depth [mm],liquid precipitation quantity [hr]
0,2021,12,31,19,0,NOAA HRRR,25.15490,21.39865,80.0,99060.0,...,798.5615,0,999999999,30.381283,0.0,0.0,NaN,0.2,0.007,1
1,2021,12,31,20,0,NOAA HRRR,27.16995,20.34902,66.2,98900.0,...,NaN,0,999999999,29.937360,0.0,0.0,NaN,0.2,0.000,1
2,2021,12,31,21,0,NOAA HRRR,28.33914,19.14065,57.5,98780.0,...,NaN,0,999999999,30.286688,0.0,0.0,NaN,0.2,0.000,1
3,2021,12,31,22,0,NOAA HRRR,28.21115,18.93496,57.1,98720.0,...,NaN,0,999999999,30.371185,0.0,0.0,NaN,0.2,0.000,1
4,2021,12,31,23,0,NOAA HRRR,26.90980,20.63275,65.3,98690.0,...,NaN,0,999999999,31.071861,0.0,0.0,NaN,0.2,0.000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13873,2023,8,1,20,0,NOAA HRRR,38.25115,18.40487,33.9,99930.0,...,NaN,0,999999999,43.355343,0.0,0.0,180.0,0.2,0.000,1
13874,2023,8,1,21,0,NOAA HRRR,39.52258,16.88270,28.6,99810.0,...,NaN,0,999999999,45.396960,0.0,0.0,180.0,0.2,0.000,1
13875,2023,8,1,22,0,NOAA HRRR,38.73403,17.70724,30.9,99760.0,...,NaN,0,999999999,44.479767,0.0,0.0,180.0,0.2,0.000,1
13876,2023,8,1,23,0,NOAA HRRR,39.80328,15.41230,25.3,99690.0,...,NaN,0,999999999,44.671688,0.0,0.0,181.0,0.2,0.000,1


In [8]:
print(amy_df.apply(lambda col: col.nunique()))

year                                                     3
month                                                   12
day                                                     31
hour                                                    24
minute                                                   1
data_flags                                               1
dry bulb temperature [C]                             13774
dew point temperature [C]                            10068
relative humidity [%]                                  898
atmospheric station pressure [Pa]                      412
extraterrestrial horizontal radiation [Wh/m^2]       13878
extraterrestrial direct normal radiation [Wh/m^2]      189
horizontal infrared radiation intensity [Wh/m^2]     13878
global horizontal radiation [Wh/m^2]                  4736
direct normal radiation [Wh/m^2]                      2284
diffuse horizontal radiation [Wh/m^2]                 1633
global horizontal illuminance [lux]                   47

In [9]:
# TODO: fix ceiling height--> it's fine actually. Just needs validation
# TODO: fix present weather codes--> was assigning series to df column
# TODO: column validation

In [12]:
# validate_df
# df, variable_properties
df = amy_df
for k,v in constants.variable_properties.items():
    if any([i in v.keys() for i in ["min", "max", "missing"]]):
        data = df.loc[:,k].values
        if "min" in v.keys() or "max" in v.keys():
            data = np.clip(data, v.get("min", None), v.get("max", None))
        if "missing" in v.keys():
            data = np.nan_to_num(data, nan=v.get("missing"), copy=False)
        df.loc[:,k] = data
df


,year,month,day,hour,minute,data_flags,dry bulb temperature [C],dew point temperature [C],relative humidity [%],atmospheric station pressure [Pa],...,ceiling height [m],present weather observation,present weather codes,precipitable water [mm],aerosol optical depth [thousandths],snow depth [cm],days since last snowfall,albedo [nondim],liquid precipitation depth [mm],liquid precipitation quantity [hr]
0,2021,12,31,19,0,NOAA HRRR,25.15490,21.39865,80.0,99060.0,...,798.5615,0,999999999,30.381283,0.0,0.0,99.0,0.2,0.007,1
1,2021,12,31,20,0,NOAA HRRR,27.16995,20.34902,66.2,98900.0,...,99999.0000,0,999999999,29.937360,0.0,0.0,99.0,0.2,0.000,1
2,2021,12,31,21,0,NOAA HRRR,28.33914,19.14065,57.5,98780.0,...,99999.0000,0,999999999,30.286688,0.0,0.0,99.0,0.2,0.000,1
3,2021,12,31,22,0,NOAA HRRR,28.21115,18.93496,57.1,98720.0,...,99999.0000,0,999999999,30.371185,0.0,0.0,99.0,0.2,0.000,1
4,2021,12,31,23,0,NOAA HRRR,26.90980,20.63275,65.3,98690.0,...,99999.0000,0,999999999,31.071861,0.0,0.0,99.0,0.2,0.000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13873,2023,8,1,20,0,NOAA HRRR,38.25115,18.40487,33.9,99930.0,...,99999.0000,0,999999999,43.355343,0.0,0.0,180.0,0.2,0.000,1
13874,2023,8,1,21,0,NOAA HRRR,39.52258,16.88270,28.6,99810.0,...,99999.0000,0,999999999,45.396960,0.0,0.0,180.0,0.2,0.000,1
13875,2023,8,1,22,0,NOAA HRRR,38.73403,17.70724,30.9,99760.0,...,99999.0000,0,999999999,44.479767,0.0,0.0,180.0,0.2,0.000,1
13876,2023,8,1,23,0,NOAA HRRR,39.80328,15.41230,25.3,99690.0,...,99999.0000,0,999999999,44.671688,0.0,0.0,181.0,0.2,0.000,1


In [20]:
from memory_profiler import memory_usage
def f():
    cache_dir = os.path.join("..", "cache")
    n_workers=1
    san_antonio = {
        "latitude" : 29.25,  # N
        "longitude" :   360-98.31,  # W
        "begin_date" : datetime.datetime(2022,1,1),
        "end_date" : datetime.datetime(2023,1,1)
    }

    # 
    dl_dates = [datetime.datetime(2022,7,30,0,0,0)]

    search_string_0h = utils.get_search_string([i.get("searchstring") for i in constants._grib_variables_0h.values()])
    search_string_1h = utils.get_search_string([i.get("searchstring") for i in constants._grib_variables_1h.values()])

    utils.get_grib_data(dl_dates,
        san_antonio.get("latitude"), 
        san_antonio.get("longitude"), 
        search_string_0h, 
        search_string_1h, 
        n_workers,
        cache_dir
    )
    
memory_usage(f)

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2022-Jul-30 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2022-Jul-29 23:00 UTC F01 ┊ GRIB2 @ aws ┊ IDX @ aws


/Users/jeh/anaconda3/envs/amy/lib/python3.8/site-packages/herbie/core.py:797: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  logic = df.search_this.str.contains(searchString)


👨🏻‍🏭 Created directory: [/Users/jeh/data/hrrr/20220730]
Note: Returning a list of [10] xarray.Datasets because cfgrib opened with multiple hypercubes.


/Users/jeh/anaconda3/envs/amy/lib/python3.8/site-packages/herbie/core.py:797: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  logic = df.search_this.str.contains(searchString)


👨🏻‍🏭 Created directory: [/Users/jeh/data/hrrr/20220729]
Note: Returning a list of [2] xarray.Datasets because cfgrib opened with multiple hypercubes.


/Users/jeh/anaconda3/envs/amy/lib/python3.8/site-packages/herbie/accessors.py:260: UserWarning: More than one time coordinate present for variable  "gribfile_projection".
  ds = ds.metpy.parse_cf()
/Users/jeh/anaconda3/envs/amy/lib/python3.8/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "tcc".
  ds = ds.metpy.assign_y_x()
/Users/jeh/anaconda3/envs/amy/lib/python3.8/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "gribfile_projection".
  ds = ds.metpy.assign_y_x()
/Users/jeh/anaconda3/envs/amy/lib/python3.8/site-packages/herbie/accessors.py:260: UserWarning: More than one time coordinate present for variable  "gribfile_projection".
  ds = ds.metpy.parse_cf()
/Users/jeh/anaconda3/envs/amy/lib/python3.8/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "unknown".
  ds = ds.metpy.assign_y_x()
/Users/jeh/anaconda3/envs/amy/li

unsupported operand type(s) for |: 'dict' and 'dict'


/Users/jeh/anaconda3/envs/amy/lib/python3.8/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "prate".
  ds = ds.metpy.assign_y_x()
/Users/jeh/anaconda3/envs/amy/lib/python3.8/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "tp".
  ds = ds.metpy.assign_y_x()
/Users/jeh/anaconda3/envs/amy/lib/python3.8/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "crain".
  ds = ds.metpy.assign_y_x()
/Users/jeh/anaconda3/envs/amy/lib/python3.8/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "cfrzr".
  ds = ds.metpy.assign_y_x()
/Users/jeh/anaconda3/envs/amy/lib/python3.8/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "cicep".
  ds = ds.metpy.assign_y_x()
/Users/jeh/anaconda3/envs/amy/lib/python3.8/site-packages/herbie/accessor

[638.34375,
 638.25,
 638.265625,
 638.28125,
 638.28125,
 638.28125,
 638.28125,
 638.28125,
 638.28125,
 638.28125,
 638.28125,
 638.28125,
 638.28125,
 638.28125,
 638.28125,
 638.28125,
 638.28125,
 638.28125,
 638.28125,
 638.28125,
 638.28125,
 638.28125,
 638.28125,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875,
 638.296875

In [ ]:
memory